# explore this [tutorial](https://plainenglish.io/blog/a-simple-guide-to-plotly-for-plotting-financial-chart-54986c996682)

In [ ]:
%pip install --user  yfinance


In [ ]:
import yfinance as yf
import pandas as pd
symbol = 'AAPL'
df = yf.download(symbol, start='2024-01-01')
dx = pd.DataFrame(df)

> [!HINT] FROM HERE
> https://pypi.org/project/yfinance/

In [9]:
# works :-)
import yfinance as yf

msft = yf.Ticker("MSFT")

# get all stock info
msft.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

## get historical market data

In [11]:
# get historical market data
import yfinance as yf

trex = yf.Ticker("TREX")

# get all stock info
trex.info

# get historical market data
trex.history(period="1mo")

{'address1': '2500 Trex Way',
 'city': 'Winchester',
 'state': 'VA',
 'zip': '22601',
 'country': 'United States',
 'phone': '540 542 6300',
 'website': 'https://www.trex.com',
 'industry': 'Building Products & Equipment',
 'industryKey': 'building-products-equipment',
 'industryDisp': 'Building Products & Equipment',
 'sector': 'Industrials',
 'sectorKey': 'industrials',
 'sectorDisp': 'Industrials',
 'longBusinessSummary': "Trex Company, Inc. manufactures and distributes composite decking, railing, and outdoor living products and accessories for residential and commercial markets in the United States. It offers decking products and accessories under the names Trex Transcend, Trex Select, Trex Signature, Trex Transcend Lineage, and Trex Enhance for protection against fading, staining, mold, and scratching; Trex Hideaway, a hidden fastening system; and Trex DeckLighting, a LED dimmable deck lighting for use on posts, floors, and steps. The company also provides Trex Transcend Railing p